In [1]:
import torch
import pandas as pd
import sys
import dotenv
import os
import torchmetrics
sys.path.append('../..')
from src.models import ResNet34,ResNet18
from torchsummary import summary
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose,ToTensor
from src.transforms import LabelMapper,make_patches,ImageResizer
from typing import Any, Tuple
from src.utils import load_model_from_folder
from tqdm.notebook import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device.type)

cuda


In [3]:
model = ResNet34(n_classes=3).to(device)

In [4]:
PATCHES_DIR = dotenv.get_key(dotenv.find_dotenv(), "PATCHES_DIR")
MODELS_DIR = dotenv.get_key(dotenv.find_dotenv(), "MODELS_DIR")
TRAIN_DIR = os.path.join(PATCHES_DIR, "train")
VAL_DIR = os.path.join(PATCHES_DIR, "val")

print(MODELS_DIR)
print(PATCHES_DIR)
print(TRAIN_DIR)
print(VAL_DIR)

/home/abdelnour/Documents/4eme_anne/S2/projet/models
/home/abdelnour/Documents/4eme_anne/S2/projet/data/patched
/home/abdelnour/Documents/4eme_anne/S2/projet/data/patched/train
/home/abdelnour/Documents/4eme_anne/S2/projet/data/patched/val


In [5]:
weights_folder = os.path.join(MODELS_DIR, "resnet34")
load_model_from_folder(model, weights_folder, verbose=True)

loading weights with name : 1710543530.8453307.pt


In [6]:
summary(model, input_size=(3,224,224), device=device.type)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [7]:
label_mapper = LabelMapper({
    0:0, # 0 is the label for benign (BY)
    1:0, 
    2:0,
    3:1, # 1 is the label for atypical (AT)
    4:1,
    5:2, # 2 is the label for malignant (MT)
    6:2,
})

In [8]:
class RoIDataset(ImageFolder):

    def __getitem__(self, index: int) -> Tuple[Any, Any]:

        x,y = super().__getitem__(index)
        path = self.imgs[index][0]
        basename = os.path.basename(path)
        name,extention = os.path.splitext(basename)
        original_roi = '_'.join(name.split('_')[:-1]) + extention

        return basename,original_roi,x,y

In [9]:
dataset = ImageFolder(
    root=VAL_DIR,
    target_transform=label_mapper,
    transform=Compose([
        ImageResizer(),
        ToTensor()
    ])
)

In [10]:
loader = DataLoader(dataset=dataset, batch_size=32)

In [13]:
def predict(
    model : nn.Module,
    dataloader : DataLoader
) -> pd.DataFrame:
    
    Y = []

    model.eval()
    
    with torch.inference_mode():
        
        for x,y in tqdm(dataloader):

            x,y = x.to(device),y.to(device)

            y_hat = model(x)
            y_hat = torch.nn.functional.softmax(x, dim=0)

            Y.append(y_hat.tolist())

    return torch.stack(Y)

In [14]:
y_hat = predict(model, loader)

  0%|          | 0/977 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
torch.randn(31000, 3).element_size()

4